In [12]:
import numpy as np
import pandas as pd

In [15]:
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.layers = [input_size] + hidden_layers + [output_size]

        self.weights = [np.random.randn(self.layers[i], self.layers[i+1]) for i in range(len(self.layers) - 1)]
        self.biases = [np.random.randn(n) for n in self.layers[1:]]

    def activation(self, x):
        return 1 / (1 + np.exp(-x))

    def activation_derivative(self, x):
        return x * (1 - x)

    def forward(self, x):
        self.activations = [x]
        self.z_values = []

        for w, b in zip(self.weights, self.biases):
            z = np.dot(self.activations[-1], w) + b
            self.z_values.append(z)
            self.activations.append(self.activation(z))

        return self.activations[-1]

    def backward(self, y_true):
        errors = [y_true - self.activations[-1]]
        deltas = [errors[-1] * self.activation_derivative(self.activations[-1])]

        for i in range(len(self.weights) - 1, 0, -1):
            error = deltas[-1] @ self.weights[i].T
            delta = error * self.activation_derivative(self.activations[i])
            deltas.append(delta)

        deltas.reverse()

        for i in range(len(self.weights)):
            self.weights[i] += self.learning_rate * np.outer(self.activations[i], deltas[i])
            self.biases[i] += self.learning_rate * deltas[i]

    def train(self, df, target_column, epochs=1000, print_every=500):
        X = df.drop(columns=[target_column]).values
        y = df[[target_column]].values

        for epoch in range(1, epochs + 1):
            total_loss = 0
            for x_sample, y_sample in zip(X, y):
                output = self.forward(x_sample)
                loss = np.mean((y_sample - output) ** 2)  # MSE
                total_loss += loss
                self.backward(y_sample)

            if epoch % print_every == 0:
                avg_loss = total_loss / len(X)
                print(f"Эпоха {epoch}, Ошибка: {avg_loss:.6f}")

    def predict(self, df):
        X = df.values
        return np.array([self.forward(x) for x in X])

In [16]:
data = {
    'feature1': [0.5, 0.2, 0.9, 0.3],
    'feature2': [0.8, 0.4, 0.1, 0.7],
    'target': [1, 0, 1, 0]
}
df = pd.DataFrame(data)

network = SimpleNeuralNetwork(input_size=2, hidden_layers=[4, 3], output_size=1)
network.train(df, target_column='target', epochs=5000, print_every=500)

predictions = network.predict(df.drop(columns=['target']))
print("Предсказания:")
print(predictions)

Эпоха 500, Ошибка: 0.246025
Эпоха 1000, Ошибка: 0.213334
Эпоха 1500, Ошибка: 0.110766
Эпоха 2000, Ошибка: 0.035091
Эпоха 2500, Ошибка: 0.014742
Эпоха 3000, Ошибка: 0.007989
Эпоха 3500, Ошибка: 0.005088
Эпоха 4000, Ошибка: 0.003589
Эпоха 4500, Ошибка: 0.002709
Эпоха 5000, Ошибка: 0.002144
Предсказания:
[[0.9360804 ]
 [0.00843706]
 [0.99427938]
 [0.06567419]]
